In [1]:
import smf2021.rank2 as rank2
import smf2021.Optimiser as O
from smf.db_engine import DbEngine
import eikon as ek
# from smf.ranker import retrieve
# from smf.ranker import rank
# from smf.ranker import add_sector_info
from smf.db_engine import DbEngine
ek.set_app_key('403255e90c7647afafbfb5c0000d60ac4c8cc536')

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf

from numpy import log
from dateutil.parser import parse
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import pmdarima as pm
import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns
from pandas.api.types import is_numeric_dtype
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import time
import datetime as dt


In [2]:
# Industrials
ind_fields = ['TR.GICSSector','TR.PE',"TR.PriceToBVPerShare",
              "TR.EPSMean", "TR.EVToSales", "TR.EVToEBIT",
              "TR.PtoEBTMeanEst", "TR.FwdPEG"]


# Comms
comms_fields = ['TR.GICSSector','TR.ROATotalAssetsPercent',
                'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
                'TR.EVToEBITDA',
                'TR.ROICActValue', 'TR.TimesInterestEarned', 'TR.FCFMeanYield',
               ]

# Healthcare
health_fields = ['TR.GICSSector','TR.PE',  'TR.EVToEBITDA',
                 'TR.EVToSales', 'TR.ROICActValue', 'TR.FCFMeanYield',
'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
                 "TR.PriceToBVPerShare"]

cons_fields = ['TR.GICSSector','TR.PE', 'TR.ROATotalAssetsPercent', 
               'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
               'TR.EVToEBITDA', 'TR.EVToSales']

fields = ['TR.GICSSector','TR.PE', "TR.PriceToBVPerShare", "TR.EPSMean",
          "TR.EVToSales", "TR.EVToEBIT",
          "TR.PtoEBTMeanEst", "TR.FwdPEG"]  

tech_fields = ['TR.GICSSector','TR.PE', "TR.PriceToBVPerShare", 
               'TR.ROICActValue', 'TR.ReinvestmentRatePct',
               'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
               'TR.FCFMeanYield']

In [3]:
ind_ranker = rank2.ranker(ind_fields)
comms_ranker = rank2.ranker(comms_fields)
health_ranker = rank2.ranker(health_fields)
cons_ranker = rank2.ranker(cons_fields)
tech_ranker = rank2.ranker(tech_fields)
all_comms = comms_ranker.get_data()
all_ind = ind_ranker.get_data()
all_health = health_ranker.get_data()
all_cons = cons_ranker.get_data()
all_tech = tech_ranker.get_data()


In [4]:
all_tech

,Instrument,GICS Sector Name,P/E (Daily Time Series Ratio),Price To Book Value Per Share (Daily Time Series Ratio),Return On Invested Capital - Actual,"Reinvestment Rate, %",ROE Common Equity %,Free Cash Flow Yield %
0,ANTO.L,Materials,17.427632,2.262901,11.45,-0.531244,6.595614,10.523560
1,AHT.L,Industrials,28.306198,6.517694,16.05,16.219474,22.249137,3.187895
2,SVT.L,Utilities,<NA>,6.240817,5.43,-3.114506,17.813969,2.673741
3,BARC.L,Financials,6.66997,0.511807,<NA>,2.07702,2.343618,NaN
4,BDEV.L,Consumer Discretionary,9.819226,1.150787,18.35,7.012702,12.824226,2.576791
...,...,...,...,...,...,...,...,...
95,AUTOA.L,Communication Services,31.956983,14.152303,29.37,26.486757,42.578711,3.997070
96,MRON.L,Industrials,<NA>,0.843106,3.25,-8.270574,-7.777626,-2.288233
97,AVST.L,Information Technology,30.399097,6.668943,18.199,0.432489,14.670011,4.528575
98,AAF.L,Communication Services,12.773553,2.060539,<NA>,5.564552,9.980863,8.801990


In [5]:
# True => lower is better.
# ROIC < WACC => remove from list.
# Interest Coverage Ratio > 1.5
ind_data = ind_ranker.industry_data('Industrials')
comms_data = comms_ranker.industry_data('Communication Services')
cons_data = cons_ranker.industry_data('Consumer Staples')
health_data = health_ranker.industry_data('Health Care')
tech_data = health_ranker.industry_data('Information Technology')
asc_ind = {'P/E (Daily Time Series Ratio)': True,
           'Price To Book Value Per Share (Daily Time Series Ratio)': True,
           'Earnings Per Share - Mean': False, 
           'Enterprise Value To Sales (Daily Time Series Ratio)': True,
           'Enterprise Value To EBIT (Daily Time Series Ratio)': False, 
           'Price / EBITDA (Mean Estimate)': True, 
           'Forward P/E/G (Daily Time Series Ratio)': True}
             
asc_comms = {'ROA Total Assets, Percent': False,
             'ROE Common Equity %': False,
             'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
             'Return On Invested Capital - Actual': False,
             'Interest Coverage Ratio': False,
             'Free Cash Flow Yield %': False}


asc_tech = {'P/E (Daily Time Series Ratio)': True,
            'ROE Common Equity %': False,
            'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
            'Enterprise Value To Sales (Daily Time Series Ratio)': True,
            'Price To Book Value Per Share (Daily Time Series Ratio)': True,

            'Return On Invested Capital - Actual': False,
            'Free Cash Flow Yield %': False}

asc_health = {'P/E (Daily Time Series Ratio)': True,
            'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
            'Enterprise Value To Sales (Daily Time Series Ratio)': True,
            'Return On Invested Capital - Actual': False,
            'Free Cash Flow Yield %': False,
            'ROE Common Equity %': False,
            'Price To Book Value Per Share (Daily Time Series Ratio)': True}

asc_cons = {'P/E (Daily Time Series Ratio)': True,
                'ROA Total Assets, Percent': False,
                'ROE Common Equity %': False,
                'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
                'Enterprise Value To Sales (Daily Time Series Ratio)': True
                }

In [6]:
ind_z_scores = ind_ranker.z_scores(ind_data, asc_ind)
health_z_scores = health_ranker.z_scores(health_data, asc_health)
comms_z_scores = comms_ranker.z_scores(comms_data, asc_comms)
tech_z_scores = tech_ranker.z_scores(tech_data, asc_tech)
cons_z_scores = cons_ranker.z_scores(cons_data, asc_cons)
cons_list = cons_z_scores.head(5)['Instrument'].tolist()
ind_list = ind_z_scores.head(5)['Instrument'].tolist()
health_list = health_z_scores.head(5)['Instrument'].tolist()
comms_list = comms_z_scores.head(5)["Instrument"].tolist()
tech_list = tech_z_scores.head(5)["Instrument"].tolist()

stocks_list = cons_list+comms_list+ind_list+health_list+tech_list
stocks_list


['IMB.L',
 'ULVR.L',
 'CCH.L',
 'TSCO.L',
 'BATS.L',
 'RMV.L',
 'AUTOA.L',
 'AAF.L',
 'VOD.L',
 'ITV.L',
 'RR.L',
 'DCC.L',
 'FERG.L',
 'RMG.L',
 'BAES.L',
 'HIK.L',
 'GSK.L',
 'SN.L',
 'AZN.L',
 'DPH.L',
 'SGE.L',
 'AVST.L',
 'HLMA.L',
 'AVV.L']

Optimiser 1

In [7]:
final_companies1 = stocks_list
data=ek.get_timeseries(final_companies1, fields='CLOSE', start_date='2021-01-19', end_date='2022-01-19' )
close_prices_12m = data.fillna(method="ffill")
close_prices_12m =close_prices_12m ._get_numeric_data()
close_prices_12m=close_prices_12m.astype(float)

In [8]:
close_prices_12m

CLOSE,IMB.L,ULVR.L,CCH.L,TSCO.L,BATS.L,RMV.L,AUTOA.L,AAF.L,VOD.L,ITV.L,...,BAES.L,HIK.L,GSK.L,SN.L,AZN.L,DPH.L,SGE.L,AVST.L,HLMA.L,AVV.L
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-23,1569.0,4144.0,2710.0,234.30,2769.0,673.0,660.2,82.25,118.84,121.00,...,552.0,2644.0,1416.0,1554.5,8480.0,4914.0,704.8,578.6,2856.0,3998.0
2021-07-26,1569.5,4031.0,2698.0,232.40,2766.0,668.2,652.2,82.45,117.60,121.25,...,550.8,2616.0,1399.8,1519.0,8292.0,4840.0,700.4,578.2,2835.0,3970.0
2021-07-27,1552.5,4051.5,2709.0,234.40,2770.5,670.8,647.0,83.15,116.74,119.35,...,553.4,2615.0,1399.2,1521.5,8253.0,4868.0,700.4,573.2,2835.0,3894.0
2021-07-28,1543.0,4049.5,2713.0,233.15,2734.0,676.0,649.6,82.50,115.34,117.75,...,560.6,2630.0,1401.6,1563.5,8266.0,4902.0,697.2,569.4,2874.0,3936.0
2021-07-29,1550.0,4107.0,2740.0,233.25,2723.0,680.2,645.4,83.65,116.40,114.15,...,572.8,2622.0,1406.4,1464.5,8277.0,4914.0,692.6,572.2,2881.0,3909.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-13,1695.5,3903.0,2687.0,289.70,2975.0,725.6,706.0,139.10,117.78,118.80,...,578.2,2061.0,1631.2,1274.0,8449.0,4290.0,800.8,602.0,2780.0,2977.0
2022-01-14,1709.5,3936.5,2645.0,285.00,3033.0,692.8,687.6,143.00,118.28,117.90,...,579.8,2056.0,1641.0,1260.5,8544.0,4200.0,798.2,603.0,2657.0,2953.0
2022-01-17,1732.0,3662.0,2663.0,289.45,3144.0,690.2,692.0,144.30,118.88,120.40,...,590.4,2069.0,1707.8,1257.5,8770.0,4226.0,810.2,602.0,2670.0,2951.0


In [9]:
min_weight, max_weight = 0.01, 0.20
print("--------------------------------------")
print(f"Weight range: [{min_weight}, {max_weight}]")
print()
mu = expected_returns.mean_historical_return(close_prices_12m)
S = risk_models.CovarianceShrinkage(close_prices_12m).ledoit_wolf()

ef = EfficientFrontier(expected_returns=mu, cov_matrix=S, weight_bounds=(min_weight, max_weight))
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

ef.portfolio_performance(verbose=True)
print(pd.Series(cleaned_weights).sort_values(ascending=False))

--------------------------------------
Weight range: [0.01, 0.2]

Expected annual return: 63.9%
Annual volatility: 10.5%
Sharpe Ratio: 5.91
TSCO.L     0.20000
AAF.L      0.20000
AVST.L     0.10827
BAES.L     0.07584
GSK.L      0.07501
BATS.L     0.05913
SGE.L      0.05694
IMB.L      0.03305
RR.L       0.02759
AZN.L      0.02417
HLMA.L     0.01000
DPH.L      0.01000
SN.L       0.01000
HIK.L      0.01000
FERG.L     0.01000
RMG.L      0.01000
ULVR.L     0.01000
DCC.L      0.01000
ITV.L      0.01000
VOD.L      0.01000
AUTOA.L    0.01000
RMV.L      0.01000
CCH.L      0.01000
AVV.L      0.01000
dtype: float64


In [10]:
print(pd.Series(cleaned_weights).sort_values(ascending=False)[:8])

TSCO.L    0.20000
AAF.L     0.20000
AVST.L    0.10827
BAES.L    0.07584
GSK.L     0.07501
BATS.L    0.05913
SGE.L     0.05694
IMB.L     0.03305
dtype: float64


In [11]:
curr_prices = ek.get_data(final_companies1, "CF_LAST")[0]
curr_prices = curr_prices.set_index("Instrument")['CF_LAST']

In [12]:
curr_prices

Instrument
IMB.L       1790.5
ULVR.L      3779.0
CCH.L       2531.0
TSCO.L       297.0
BATS.L      3269.0
RMV.L        634.8
AUTOA.L      647.8
AAF.L        144.0
VOD.L       138.44
ITV.L       123.25
RR.L        122.72
DCC.L       6486.0
FERG.L     11225.0
RMG.L        440.6
BAES.L       597.0
HIK.L       2005.0
GSK.L       1641.0
SN.L        1237.5
AZN.L       8650.0
DPH.L       3890.0
SGE.L        690.6
AVST.L       624.0
HLMA.L      2429.0
AVV.L       2878.0
Name: CF_LAST, dtype: Float64

In [13]:
da = DiscreteAllocation(cleaned_weights, curr_prices, total_portfolio_value=60000)
allocation, leftover = da.lp_portfolio()
print("Left Over: {}".format(leftover))

Left Over: 72.11000000000058


In [14]:
df = pd.DataFrame()
df['Number of shares'] = pd.Series(allocation)
df['Weights'] = pd.Series(cleaned_weights).sort_values(ascending=False)
df['Current Price'] = pd.Series(curr_prices)
df

,Number of shares,Weights,Current Price
IMB.L,1,0.03305,1790.5
TSCO.L,41,0.20000,297.0
BATS.L,1,0.05913,3269.0
RMV.L,1,0.01000,634.8
AUTOA.L,1,0.01000,647.8
AAF.L,83,0.20000,144.0
VOD.L,4,0.01000,138.44
ITV.L,5,0.01000,123.25
RR.L,14,0.02759,122.72
RMG.L,2,0.01000,440.6


In [15]:
(df['Number of shares'] * df['Current Price']).sum()
# df.to_csv("optimisation1.csv")

59927.89